In [1]:
# imports
import requests
import sqlalchemy as db
import pandas as pd
from bs4 import BeautifulSoup as bs
from alchemyDB import *

In [2]:
# make DB?
engine = db.create_engine('sqlite:///bgg.sqlite')

connection = engine.connect()

metadata = db.MetaData()

Base.metadata.create_all(engine)

In [10]:
Game.__table__

Table('games', MetaData(bind=None), Column('id', Integer(), table=<games>, primary_key=True, nullable=False), Column('name', String(), table=<games>), Column('description', String(), table=<games>), Column('published', Integer(), table=<games>), Column('minplayers', Integer(), table=<games>), Column('maxplayers', Integer(), table=<games>), Column('suggested_numplayers', Integer(), table=<games>), Column('playingtime', Integer(), table=<games>), Column('minplaytime', Integer(), table=<games>), Column('maxplaytime', Integer(), table=<games>), Column('minage', Integer(), table=<games>), Column('suggested_playerage', Integer(), table=<games>), Column('language_dependence', Boolean(), table=<games>), Column('designer', String(), table=<games>), schema=None)

Test 1 request pull

In [11]:
base = 'https://boardgamegeek.com/xmlapi2/thing?id' # ex: =30549&type=boardgame,boardgameexpansion'

In [12]:
req=requests.get(base+'=1&type=boardgame,boardgameexpansion')

In [13]:
soup = bs(req.content, 'xml')

In [28]:
# need to find way to loop through poll and find winning value
soup.find_all('poll')

[<poll name="suggested_numplayers" title="User Suggested Number of Players" totalvotes="125">
 <results numplayers="1">
 <result numvotes="0" value="Best"/>
 <result numvotes="1" value="Recommended"/>
 <result numvotes="78" value="Not Recommended"/>
 </results>
 <results numplayers="2">
 <result numvotes="0" value="Best"/>
 <result numvotes="1" value="Recommended"/>
 <result numvotes="79" value="Not Recommended"/>
 </results>
 <results numplayers="3">
 <result numvotes="1" value="Best"/>
 <result numvotes="25" value="Recommended"/>
 <result numvotes="70" value="Not Recommended"/>
 </results>
 <results numplayers="4">
 <result numvotes="23" value="Best"/>
 <result numvotes="81" value="Recommended"/>
 <result numvotes="9" value="Not Recommended"/>
 </results>
 <results numplayers="5">
 <result numvotes="106" value="Best"/>
 <result numvotes="12" value="Recommended"/>
 <result numvotes="2" value="Not Recommended"/>
 </results>
 <results numplayers="5+">
 <result numvotes="1" value="Best"/

In [29]:
# testing schema for adding a game from soup
game = Game(
    id = 1,
    name=soup.find('name')['value'],
    description=soup.find('description').text,
    published=int(soup.find('yearpublished')['value']),
    minplayers=int(soup.find('minplayers')['value']),
    maxplayers=int(soup.find('maxplayers')['value']),
    #suggested_numplayers,
    playingtime=int(soup.find('playingtime')['value']),
    minplaytime=int(soup.find('minplaytime')['value']),
    maxplaytime=int(soup.find('maxplaytime')['value']),
    minage=int(soup.find('minage')['value'])
    #,suggested_playerage=,
    #language_dependence=
    #,designer=""
)

In [33]:
# connect to DB?
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session=Session()

In [55]:
session.rollback()

In [57]:
# Not tested yet
for i in range(1,501):
    req = requests.get(base +f'={i}&type=boardgame,boardgameexpansion')
    soup = bs(req.content, 'xml')
    if soup.find('name'):
        game = Game(
            id = i,
            name=soup.find('name')['value'],
            description=soup.find('description').text,
            published=int(soup.find('yearpublished')['value']),
            minplayers=int(soup.find('minplayers')['value']),
            maxplayers=int(soup.find('maxplayers')['value']),
            #suggested_numplayers,
            playingtime=int(soup.find('playingtime')['value']),
            minplaytime=int(soup.find('minplaytime')['value']),
            maxplaytime=int(soup.find('maxplaytime')['value']),
            minage=int(soup.find('minage')['value'])
            #,suggested_playerage=,
            #language_dependence=
            #,designer=""
        )
        session.add(game)
        session.commit()
    

In [41]:
from sqlalchemy import inspect

inspector = inspect(engine)

In [42]:
print(inspector.get_table_names())

['games']


In [43]:
print(inspector.get_columns('games'))

[{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}, {'name': 'name', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'description', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'published', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'minplayers', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'maxplayers', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'suggested_numplayers', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'playingtime', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'minplaytime', 'type': I

In [58]:
df = pd.read_sql_query("SELECT * FROM games", engine)
df.tail()

,id,name,description,published,minplayers,maxplayers,suggested_numplayers,playingtime,minplaytime,maxplaytime,minage,suggested_playerage,language_dependence,designer
80,391,Ocean,The idea is for players to populate the variou...,1999,2,6,None,20,20,20,8,None,None,None
81,431,Alhambra: The Card Game,Alhambra: The Card Game is a game that both in...,1992,3,6,None,45,45,45,10,None,None,None
82,451,Beutelschneider,"Translated as Cutpurse, this is a fairly uniqu...",1997,3,4,None,30,30,30,0,None,None,None
83,470,Road to the White House,extrapolated from the preview:&#10;&#10;The es...,1992,3,6,None,240,240,240,12,None,None,None
84,490,Warangel,"Warangel is a light, fantasy wargame, comprisi...",2000,2,6,None,60,60,60,12,None,None,None
